In [2]:
import cv2
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16
# Load the vgg16 model
model=vgg16.VGG16(weights='imagenet', include_top = False,input_shape = (244,244,3))
# print layers
for (i,layer) in enumerate(model.layers):
    print(str(i)+"  "+model.layers[0].__class__.__name__+"  "+str(model.layers[i].trainable))
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False

58892288/58889256 [==============================] - 183s 3us/step
0  InputLayer  False
1  InputLayer  True
2  InputLayer  True
3  InputLayer  True
4  InputLayer  True
5  InputLayer  True
6  InputLayer  True
7  InputLayer  True
8  InputLayer  True
9  InputLayer  True
10  InputLayer  True
11  InputLayer  True
12  InputLayer  True
13  InputLayer  True
14  InputLayer  True
15  InputLayer  True
16  InputLayer  True
17  InputLayer  True
18  InputLayer  True


In [3]:
def lw(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 3

FC_Head = lw(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = (r'C:/Users/chakri/Desktop/mlops/train/') 
validation_data_dir = (r'C:/Users/chakri/Desktop/mlops/test/')


# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.5,
      height_shift_range=0.5,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 324 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [8]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import PIL.Image
                     
checkpoint = ModelCheckpoint("myface_recognition.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 337
nb_validation_samples = 25

# We only train 5 EPOCHS 
epochs = 3
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)
model.save("myface_recognition.h5")

Epoch 1/3
21/21 [==============================] - 219s 10s/step - loss: 0.3096 - accuracy: 0.8782 - val_loss: 0.6338 - val_accuracy: 0.7467

Epoch 00001: val_loss improved from inf to 0.63378, saving model to myface_recognition.h5
Epoch 2/3
21/21 [==============================] - 226s 11s/step - loss: 0.2344 - accuracy: 0.8956 - val_loss: 0.7924 - val_accuracy: 0.7333

Epoch 00002: val_loss did not improve from 0.63378
Epoch 3/3
21/21 [==============================] - 236s 11s/step - loss: 0.2329 - accuracy: 0.9015 - val_loss: 0.8130 - val_accuracy: 0.7200

Epoch 00003: val_loss did not improve from 0.63378
